In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
from numpy.linalg import solve, norm
from scipy.integrate import quad_vec
from scipy.integrate import quad
from numpy.linalg import matrix_rank
import math
import scipy

The general Poisson problem. Find $u:[a,b]\subset\Reals\rightarrow\Reals$ such that:

\begin{align*}
 -\bigl(-D(x)\, u'\bigr)'(x) + S(x)\,u(x) + f(x) &= 0 \quad\quad \forall \quad\quad x\in [a,b], \\
\end{align*}
        with some defined boundary conditions
   
    For the current 1D slab reflected reactor, the equations for the core and reflector regions can be defined as:
        Core Region: 
       
\begin{align*}
 u''(x) + B^2\,u(x) &= 0 \quad\quad \forall \quad\quad x\in [0,a/2], \\
\end{align*}


        
         and for the reflector region: 
         
\begin{align*}
  u''(x) - L^{-2}\,u(x) &= 0 \quad\quad \forall \quad\quad x\in [a/2,h], \\
\end{align*}

Comparing the equations for the core and reflector with the poissons equation, we find that

\begin{align*}
D = 1, \\
f(x) &= 0 \\
S(x) = B^2 ; for core region \\
      =L^{-2};in the reflector region \\
\end{align*}
        
         with boundary conditions:
         u'(0) = 0 
         u(h) = 0

In [ ]:
'''Domain'''

x_a = 0
x_b = 100
a=20
source_slope_value = 4.13e-4
l_2= 94.70
diff_coeff = 1

In [ ]:
'''Domain partition'''
from fem_functions import get_domain_partition

In [ ]:
'''Parent mapping'''
from fem_functions import get_parent_mapping

In [ ]:
'''Parent basis functions'''
from fem_functions import get_global_basis_functions

In [ ]:
'''Any global basis function'''

In [ ]:
'''All global basis functions'''

In [ ]:
'''Defined Functions'''


def global_basis_function(i, x, domain_partition, parent_mapping, parent_basis_functions):
    """Evaluate the ith global FE basis function and its derivative on x points.
    
    This is never needed in practice. It is here for demonstrating the theory.
    """
    try:
        len(x)
    except TypeError:
        x = np.array([x])
  
    if not isinstance(x, np.ndarray):
       assert isinstance(x, list) or isinstance(x, tuple)
       x = np.array(x)
  
    phi_i_x = np.copy(x) * 0.0 # initialization
    phi_prime_i_x = np.copy(x) * 0.0 # initialization
        
    patches = domain_partition[0]
    local_to_global_node_id_map = domain_partition[2]
    inverse_parent_mapping = parent_mapping_prime[2]
    inverse_parent_mapping_prime = parent_mapping[3]
    parent_basis_func_list = parent_basis_functions[0]
    parent_basis_func_prime_list = parent_basis_functions[1]
  
    # expensive reverse lookup
    for j, x_j in enumerate(x):
        for e, nodes_x in enumerate(patches):
            if nodes_x[0] <= x_j <= nodes_x[1]:
                n_lnodes = len(nodes_x)
                for I in range(n_lnodes):
                    if local_to_global_node_id_map[e][I] == i:
                        x_e_bar = (nodes_x[0] + nodes_x[1])/2 
                        h_e = nodes_x[1] - nodes_x[0]
                        zetta = inverse_parent_mapping(x_j, x_e_bar, h_e)
                        d_zetta = inverse_parent_mapping_prime(h_e)
                        
                        phi_i_x[j] = parent_basis_func_list[I](zetta)
                        phi_prime_i_x[j] = parent_basis_func_prime_list[I](d_zetta)
                break
    return [phi_i_x, phi_prime_i_x]

def get_global_basis_functions(domain_partition, parent_mapping, parent_basis_functions, global_basis_function):
    
    basis_func_list = list()
    basis_func_prime_list = []
    n_gnodes = domain_partition[1].size

    local_to_global_node_id_map = domain_partition[2]

    phi_i = lambda i, x: global_basis_function(i,x, domain_partition, parent_mapping, parent_basis_functions)[0]
    phi_prime_i = lambda i, x: global_basis_function(i,x, domain_partition, parent_mapping, parent_basis_functions)[1]

    visited = [False]*n_gnodes
    for e in range(n_elem):
        for I in range(len(local_to_global_node_id_map[e])):
            gnode_id = local_to_global_node_id_map[e][I]
            if gnode_id >= 0 and not visited[gnode_id]:
                      basis_func_list.append(lambda x, i=gnode_id: phi_i(i,x))
                      basis_func_prime_list.append(lambda x, i=gnode_id: phi_prime_i(i,x))
                      visited[gnode_id] = True

    assert len(basis_func_list) >= 1, 'There are no basis functions to build.'

    return [basis_func_list, basis_func_prime_list]

def plot(title='Lagrange Basis Functions'):
    
    import matplotlib.pyplot as plt
    #%matplotlib inline
    plt.style.use('classic')
    plt.figure(1, figsize=(14, 5))

    npts = 200
    x_pts = np.linspace(x_a, x_b, npts)
    for (i,phi_i) in enumerate(phi_list):
        plt.plot(x_pts, phi_i(x_pts),'-',label=r'$\phi_%i$'%i)

    gnodes_x = domain_partition[1]
    plt.scatter(gnodes_x, np.zeros(gnodes_x.size), color='red', marker='x', s=80, label='nodes')

    plt.title(title, fontsize=20)
    plt.ylabel(r'$\phi_i(x)$', fontsize=18)
    plt.xlabel(r'$x$', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.legend(loc='best',fontsize=12)
    plt.grid(True)
    plt.show()

def FourierBasis(x,N,shift_fourier,Kappa):
    function=np.zeros((len(x),2*N+1))
    for i in range (len(x)):
        for j in range((2*N+1)):
            if (j==0):
                function[i,j]=1
            elif (j%2)==1:
                function[i,j]=np.cos(((j//2)+1)*Kappa*(x[i]-shift_fourier))
            elif (j%2)==0:
                function[i,j]=np.sin((j//2)*Kappa*(x[i]-shift_fourier))
    return np.array(function)

def Plot(function,x_pts):
    f = plt.figure()
    f.set_figwidth(20)
    f.set_figheight(5)
    plt.style.use('classic')
    for i in range(len(function[0])):
        if i==0:
            plt.plot(x_pts,function[:,0],label='1')
        elif i%2==1:
            plt.plot(x_pts,function[:,i], label=r'$cos$(%ik(${x}$ - $\bar{x}$))'%(((i/2)+0.5)))
        elif i % 2 == 0:
            plt.plot(x_pts,function[:,i], label=r'$sin$(%ik(${x}$ - $\bar{x}$))'%(i/2))
        for _ in range(len(function[0])//2):
            plt.legend( bbox_to_anchor=(1.05, 1.0),loc='best')
    plt.xlabel(r'$X$',fontsize = 19)
    plt.ylabel(r'$Cos x, Sin x$',fontsize=19)
    plt.title('Fourier basis func at Mode = %i'%N, fontsize=20)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    return plt.show()


In [ ]:
'''Domain'''

x_a = 0
x_b = 200

In [ ]:
'''Parameters and data'''

#diff_coeff = 0.1 #D
#source_slope_value = -1e-2

diff_coeff = 1
source_bias_value = 0
source_slope_value = 4.13e-4

u_a_0 = 3.5  # initial value
u_b_0 = 3.5  # initial value

In [ ]:
'''Generate the scaffolding of the manufactured solution'''

shape_pts_y = [u_a_0,10,-7,-5,9,-8,28,-23,12,1,10,15,18,u_b_0]
shape_pts_x = [x for x in np.linspace(x_a, x_b, len(shape_pts_y))]
shape_pts = list(zip(shape_pts_x, shape_pts_y))

def TargetFunction(x,y,markersize):
    f = plt.figure()
    f.set_figwidth(15)
    f.set_figheight(5)
    plt.style.use('classic')
    plt.plot(x,y)
    p1=plt.scatter(x,y,marker = '*',s=markersize,edgecolors = 'black', linewidth=.5,c = 'red')
    plt.legend([p1],['Share Points'],scatterpoints=2,loc ="upper right")
    plt.xlabel('X')
    plt.ylabel('f(X)')
    plt.title('Target Function', fontsize=20)
    plt.rcParams.update({'font.size': 18})
    plt.grid()
    return plt


f = TargetFunction(shape_pts_x,shape_pts_y,markersize = 50)

In [ ]:
'''Build the basis functions list'''
m= 300
shift_fourier=0
wavelength = x_b -x_a
kappa = 2*math.pi/wavelength
x_tilde_pts = np.linspace(x_a,x_b,m)
u = interpolate.interp1d(shape_pts_x, shape_pts_y, kind='linear',fill_value='array-like')


N=10

fb = FourierBasis(x_tilde_pts,N,shift_fourier,kappa)
Plot(fb,x_tilde_pts)


In [ ]:
'''Functions for Gram Matrix'''

def index_hand_fourier(p):
    t0=0
    if p>=1:
        for _ in range(p):
            t0 += 2*N+1
    return t0

def FourierBasis_single(x_pts,j,K,shift_fourier):
    if (j==0):
        mat=1
    elif (j%2)==1:
        mat=np.cos(((j//2)+1)*K*(x_pts-shift_fourier))
    elif (j%2)==0:
        mat=np.sin((j//2)*K*(x_pts-shift_fourier))
    return mat

def derivative_FourierBasis_single(x_pts,j,K,shift_fourier):
    if (j==0):
        mat = 0
    elif (j%2)==1:
        mat = -(((j//2)+1)*K*np.sin(((j//2)+1)*K*(x_pts-shift_fourier)))
    elif (j%2)==0:
        mat = (((j//2)+1)*K*np.cos((j//2)*K*(x_pts-shift_fourier)))
    return mat

def FourierBasis_matrix(x_pts,N,K,shift_fourier):
    matrix = np.zeros((len(x_pts),2*N+1))
    for i in range (len(x_pts)):
        for j in range((2*N+1)):
            matrix = FourierBasis_single(x_pts[i],j,K,shift_fourier)
    return np.array(matrix)

def derivative_g_c(x,i,j,K,shift_fourier):
    return derivative_FourierBasis_single(
        x, i, K, shift_fourier
    ) * derivative_FourierBasis_single(x, j, K, shift_fourier)

def g_c(x,i,j,K,shift_fourier):
    return FourierBasis_single(x, i, K, shift_fourier) * FourierBasis_single(
        x, j, K, shift_fourier
    )

def f_das(x):
    dx = np.array(10**(-6))
    return ((u(x+dx)-u(x))/dx)

#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++#
def Fourierbasis_gram_matrix(N,K,x_min,x_max,shift_fourier):
    gram_matrx_con = np.zeros(((np.sum(2*N)+1),(np.sum(2*N)+1)))
    for p in range(1):
        for i in range(2*N+1):
            for j in range(2*N+1):
                gram_matrx_con[int(index_hand_fourier(p)+i),int(index_hand_fourier(p)+j)] = (quad(g_c,x_min,x_max,args = (i,j,K,shift_fourier), limit = 10000)[0])+g_c(x_min,i,j,K,shift_fourier)+g_c(x_max,i,j,K,shift_fourier)+derivative_g_c(x_min,i,j,K,shift_fourier)+derivative_g_c(x_max,i,j,K,shift_fourier)
    return gram_matrx_con

def Load_vec_con(x,i,K,shift_fourier):
    return (FourierBasis_single(x,i,K,shift_fourier)*u(x))

def derivative_Load_vec_con(x,i,K,shift_fourier):
    return (derivative_FourierBasis_single(x,i,K,shift_fourier)*u(x))

def Fourier_load_vector(N,K,x_min,x_max,shift_fourier):
    b_vec_c =np.zeros((np.sum(2*N)+1))
    for i in range(2*N + 1):
        b_vec_c[i] = (quad(Load_vec_con, x_min, x_max, args = (i,K,shift_fourier), limit = 10000)[0])+Load_vec_con(x_min,i,K,shift_fourier)+Load_vec_con(x_max,i,K,shift_fourier)+derivative_Load_vec_con(x_min,i,K,shift_fourier)+derivative_Load_vec_con(x_max,i,K,shift_fourier)
    return b_vec_c

def bestg_vec_func_con(x,N,c_star_vec_con,K,shift_fourier):
    a_mtrx = FourierBasis(x,N,shift_fourier,K)
    return a_mtrx@c_star_vec_con

In [ ]:
'''Build the Gram matrix'''

fourierbasis_gram_matrix = Fourierbasis_gram_matrix(N,kappa,x_a,x_b,shift_fourier)


In [ ]:
'''Build load vector'''

fourier_load_vector = Fourier_load_vector(N,kappa,x_a,x_b,shift_fourier)


if fourierbasis_gram_matrix.shape[0] > fourierbasis_gram_matrix.shape[1]:
    print('G is overdetermined.')
elif fourierbasis_gram_matrix.shape[0] < fourierbasis_gram_matrix.shape[1]:
    print('G is underdetermined.')  
else:
    print('G is determined.')

if matrix_rank(fourierbasis_gram_matrix) == min(fourierbasis_gram_matrix.shape):
    print('G is full rank.')
else:
    print('G is rank deficient.')

In [ ]:
'''Compute optimal coefficient vector'''

beta_vec = solve(fourierbasis_gram_matrix,fourier_load_vector)

g_best_vec_1=bestg_vec_func_con(x_tilde_pts,N,beta_vec,kappa,shift_fourier)
#print(g_best_vec_1)

In [ ]:
def FourierBasis_Mat(x,N,shift_fourier,Kappa):
    function=np.zeros((len(x),2*N+1))
    for i in range (len(x)):
        for j in range((2*N+1)):
            if (j==0):
                function[i,j]=1
            elif (j%2)==1:
                function[i,j]=np.cos(((j//2)+1)*Kappa*(x[i]-shift_fourier))
            elif (j%2)==0:
                function[i,j]=np.sin((j//2)*Kappa*(x[i]-shift_fourier))
    return np.array(function)

def Derivative_FourierBasis_Mat(x,N,shift_fourier,Kappa):
    function = np.zeros((len(x),2*N+1))
    for i in range (len(x)):
        for j in range((2*N+1)):
            if (j==0):
                function[i,j]=0
            elif (j%2)==1:
                function[i,j]=-(((j//2)+1)*Kappa*np.sin(((j//2)*Kappa*(x[i]-shift_fourier))))
            elif (j%2)==0:
                function[i,j]=(((j//2)+1)*Kappa*np.cos((j//2)*Kappa*(x[i]-shift_fourier)))
    return np.array(function)

def Derivative2_FourierBasis_Mat(x,N,shift_fourier,Kappa):
    function = np.zeros((len(x),2*N+1))
    for i in range (len(x)):
        for j in range((2*N+1)):
            if (j==0):
                function[i,j]=0
            elif (j%2)==1:
                function[i,j]=-(((((j//2)+1)*Kappa)**2)*np.cos(((j//2)*Kappa*(x[i]-shift_fourier))))
            elif (j%2)==0:
                function[i,j]=-(((((j//2)+1)*Kappa)**2)*np.sin((j//2)*Kappa*(x[i]-shift_fourier)))
    return np.array(function)

def evaluation_matrix(x,N,shift_fourier,Kappa):
    return FourierBasis_Mat(x,N,shift_fourier,Kappa)

def evaluation_matrix_derivative1(x,N,shift_fourier,Kappa):
    return Derivative_FourierBasis_Mat(x,N,shift_fourier,Kappa)

def evaluation_matrix_derivative2(x,N,shift_fourier,Kappa):
    return Derivative2_FourierBasis_Mat(x,N,shift_fourier,Kappa)


In [ ]:
'''Build the best approximant function'''

def u_manufactured(x,N,shift_fourier,Kappa):
    return evaluation_matrix(x,N,shift_fourier,Kappa)@beta_vec

def u_prime_manufactured(x,N,shift_fourier,Kappa):
    return evaluation_matrix_derivative1(x,N,shift_fourier,Kappa)@beta_vec
    
def diff_flux_x_manufactured(x,N,shift_fourier,Kappa):
    return -diff_coeff*u_prime_manufactured(x,N,shift_fourier,Kappa)


In [ ]:
'''Plot comparison of '''

n_pts = 300
x_pts = np.linspace(x_a,x_b,n_pts)

u_manufactur = u_manufactured(x_pts,N,shift_fourier,kappa)
print(u_manufactured)

def Plot(g_best_vec_1,shape_pts_x,shape_pts_y,x_pts):
    f = plt.figure()
    f.set_figwidth(20)
    f.set_figheight(5)
    plt.style.use('classic')
    
    p1=plt.plot(shape_pts_x,shape_pts_y,'b--',label='f')
    p2=plt.plot(x_pts,g_best_vec_1,'r-',label='g')

    
    plt.title(r'Manufactured Function $u_\mathrm{mfac}$ ($m\rightarrow\infty$, n='+str(N)+')',fontsize=20)
    plt.legend(loc='best')
    plt.xlabel('$x$',fontsize = 19)
    plt.ylabel('$f(x),g(x)$',fontsize = 19)
    plt.rcParams.update({'font.size': 20})
    plt.grid()
    return plt.show()

plot_fourier_best_manfactured = Plot(u_manufactur,shape_pts_x,shape_pts_y,x_pts) 


In [ ]:

print('u_mfac(x_a) = ', u_manufactured([x_a]*n_pts,N,shift_fourier,kappa)[0], '; u_a_0 = ', u_a_0)
print('u_mfac(x_b) = ', u_manufactured([x_b]*n_pts,N,shift_fourier,kappa)[0], '; u_b_0 = ', u_b_0)

In [ ]:
print("u'_mfac(x_a) = ", u_prime_manufactured([x_a]*n_pts,N,shift_fourier,kappa)[0])
print("u'_mfac(x_b) = ", u_prime_manufactured([x_b]*n_pts,N,shift_fourier,kappa)[0])

In [ ]:
'''Update Boundary Condition Values'''

u_a = u_manufactured([x_a]*n_pts,N,shift_fourier,kappa)[0]
u_b = u_manufactured([x_b]*n_pts,N,shift_fourier,kappa)[0]

In [ ]:
'''Build the best approximant function'''

def u_prime2_manufactured(x,N,shift_fourier,Kappa):
    return evaluation_matrix_derivative2(x,N,shift_fourier,Kappa)@beta_vec

n_pts = 500

x = np.linspace(x_a, x_b, n_pts)

fin=u_manufactured(x,N,shift_fourier,kappa)

fin2 = u_prime2_manufactured(x,N,shift_fourier,kappa)
#print(fin.shape,'\n',fin2.shape)


In [ ]:
a=25
source_slope_value = 4.13e-4
l_2= 9.470
h = x_b
x_n=100


def source_slope(x,a,source_slope_value):
    if isinstance(x,(list,np.ndarray)):
        source_s=np.zeros(len(x))
        for i, j in enumerate(x):
            source_s[i] = source_slope_value if (j<=a) else -1/(l_2**2)
    return source_s

x = np.linspace(x_a,x_b,100)
slope_x = source_slope(x,a,source_slope_value)

plt.figure(1, figsize=(14, 5))
plt.plot(x,slope_x,label = 'Source Slope')
plt.title(r'Source Slope s$(x)$',fontsize = 20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(loc='best', fontsize=12)
plt.grid(True)
plt.xlabel(r'$cm$')
plt.ylabel(r'$s(x)$')



In [ ]:
def Derivative2_FourierBasis_Mat(x,N,shift_fourier,Kappa):
    function = np.zeros((len([x]),2*N+1))
    for i in range (len([x])):
        for j in range((2*N+1)):
            if (j==0):
                function[i,j]=0
            elif (j%2)==1:
                function[i,j]=-(((((j//2)+1)*Kappa)**2)*np.cos(((j//2)*Kappa*(x[i]-shift_fourier))))
            elif (j%2)==0:
                function[i,j]=-(((((j//2)+1)*Kappa)**2)*np.sin((j//2)*Kappa*(x[i]-shift_fourier)))
    return np.array(function)

In [ ]:
'''Build the best approximant function'''

# def source_bias(x):
#     slope_new =source_slope(x_n,a,source_slope_value)
#     manu_prime=u_prime2_manufactured(x,N,shift_fourier,kappa)
#     manu =u_manufactured(x,N,shift_fourier,kappa)
#     a_m = np.zeros(len(manu))
#     for i in range(len(manu)):
#         for j in range(len(slope_new)):
#             a_m[j] = -diff_coeff *manu_prime[i]  - slope_new[j] * manu[i]
#     return a_m
def source_bias(x):
    return -diff_coeff * u_prime2_manufactured(x,N,kappa,shift_fourier) - source_slope(x,a,source_slope_value) * u_manufactured(x,N,kappa,shift_fourier)

s = source_bias(x)

plt.figure(1, figsize=(14, 5))
plt.plot(x,s,label = 'Source Slope')
plt.title(r'Source bias s$(x)$',fontsize = 20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(loc='best', fontsize=12)
plt.grid(True)
plt.xlabel(r'$cm$')
plt.ylabel(r'$s(x)$')
plt.show()


In [ ]:
'''Source function'''


# n_pts = 500
# x = np.linspace(x_a, x_b, n_pts)
# f_values = source_bias(x,N,shift_fourier,kappa)
# s_u_values = source_slope_value * u_manufactured(x,N,shift_fourier,kappa)
# import matplotlib.pyplot as plt

# (fig, ax1) = plt.subplots(1, figsize=(14, 5))
# ax1.plot(x, f_values, 'r-', label='Source Bias function')
# ax1.set_xlabel(r'$x$ [cm]', fontsize=18)
# ax1.set_ylabel(r'$f(x)$', fontsize=18, color='red')
# ax1.tick_params(axis='y', labelcolor='red', labelsize=16)
# ax1.tick_params(axis='x', labelsize=16)
# ax1.legend(loc='upper left', fontsize=12)
# ax1.grid(True)

# ax2 = ax1.twinx()
# #ax2.plot(x, s_u_values,'-', color='blue', label='Source linear term')
# ax2.set_ylabel(r"$S(x)\, u_\mathrm{mfac}(x)$", fontsize=16, color='blue')
# ax2.tick_params(axis='y', labelcolor='blue', labelsize=16)
# ax2.legend(loc='upper right', fontsize=12)

# ax3 = ax1.twinx()
# ax3.plot(x, s_u_values+f_values,'-', color='black', label='Source')
# ax3.set_ylabel(r"$S(x)\, u_\mathrm{mfac}(x) + f(x)$", fontsize=16, color='black')
# ax3.tick_params(axis='y', labelcolor='black', labelsize=16)
# ax3.legend(loc='upper center', fontsize=12)
# ax3.spines["right"].set_position(("axes", 1.125))

# plt.title(r'Manufactured Source', fontsize=20)
# plt.show()


In [ ]:
def convertToNumpy(inputData, twoOutputs=False):
    numpyArray = np.zeros((len(inputData[:]), len(inputData[0])))
    for tupCount, tup in enumerate(inputData[:]):
        for eleCount, ele in enumerate(tup):
            numpyArray[tupCount][eleCount] = ele
    if twoOutputs == True:
        return (numpyArray[:, 0], numpyArray[:, 1])
    else:
        return (numpyArray)

# TODO Rename this here and in `get_parent_basis_functions`
def _extracted_from_get_parent_basis_functions_10(parent_basis_func_list, parent_basis_func_prime_list):
    parent_basis_func_list.append(lambda zetta: -((zetta - 1) / 2) * (-zetta))
    parent_basis_func_list.append(lambda zetta: -(zetta - 1) * (zetta + 1))
    parent_basis_func_list.append(lambda zetta: (zetta + 1) / 2 * zetta)

    parent_basis_func_prime_list.append(lambda zetta: (2 * zetta - 1) / 2)
    parent_basis_func_prime_list.append(lambda zetta: -2 * zetta)
    parent_basis_func_prime_list.append(lambda zetta: (2 * zetta + 1) / 2)

def plot_func(domain_partition, phi_list, x_min,x_max,title='Lagrange Basis Functions'):
    import matplotlib.pyplot as plt

    plt.style.use('classic')
    plt.figure(1, figsize=(14, 5))

    npts = 500
    x_pts = np.linspace(x_min, x_max, npts)
    # x_pts = domain_partition[1]
    for (i, phi_i) in enumerate(phi_list):
        plt.plot(x_pts, phi_i(x_pts), '-', label=r'$\phi_{%i}$' % i)

    gnodes_x = domain_partition[1]
    plt.scatter(gnodes_x, np.zeros(gnodes_x.size), color='red', marker='x', s=80, label='nodes')

    plt.title(title, fontsize=20)
    plt.ylabel(r'$\phi_i(x)$', fontsize=18)
    plt.xlabel(r'$x$', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.legend(loc='best', fontsize=12)
    plt.grid(True)
    plt.show()
    
def global_basis_deriv(i, x, domain_partition, parent_mapping, parent_basis_functions):
    """Evaluate the ith global FE basis function and its derivative on x points.

    This is never needed in practice. It is here for demonstrating the theory.
    """

    try:
        len(x)
    except TypeError:
        x = np.array([x])

    if not isinstance(x, np.ndarray):
        assert isinstance(x, (list, tuple))
        x = np.array(x)

    phi_i_x = np.copy(x) * 0.0  # initialization
    phi_prime_i_x = np.copy(x) * 0.0  # initialization

    patches = domain_partition[0]
    local_to_global_node_id_map = domain_partition[2]
    inverse_parent_mapping = parent_mapping[2]
    parent_basis_func_list = parent_basis_functions[0]
    parent_basis_deriv_list = parent_basis_functions[1]
    n_elem = len(patches)
    # expensive reverse lookup
    for j, x_j in enumerate(x):
        for e, nodes_x in enumerate(patches):
            if nodes_x[0] <= x_j <= nodes_x[1]:
                # n_lnodes = len(nodes_x)
                n_lnodes = len(parent_basis_func_list)
                for I in range(n_lnodes):
                    if local_to_global_node_id_map[e][I] == i:
                        x_e_bar = (nodes_x[0] + nodes_x[1]) / 2
                        h_e = nodes_x[1] - nodes_x[0]
                        zetta = inverse_parent_mapping(x_j, x_e_bar, h_e)

                        phi_prime_i_x[j] = parent_basis_deriv_list[I](zetta)
                break
    return phi_prime_i_x

def assemble_gram_matrix(a_mtrx, domain_partition, parent_mapping, parent_basis_func_list):
    n = len(parent_basis_func_list[0])

    local_to_global_node_id_map = domain_partition[2]

    local_a_mtrx = np.zeros((n, n), dtype=np.float64)

    for I in range(n):
        parent_basis_func_I = parent_basis_func_list[0][I]
        for J in range(n):
            integrand = lambda zeta: parent_basis_func_I(zeta) * parent_basis_func_list[0][J](zeta)
            (local_a_mtrx[I, J], _) = scipy.integrate.quad(integrand, -1, 1)
    for (e, gnode_ids) in enumerate(local_to_global_node_id_map):
        (x_0, x_1) = domain_partition[0][e]
        h_e = x_1 - x_0
        parent_mapping_jacobian = parent_mapping[1](h_e)
        for (I, i) in enumerate(gnode_ids):
            for (J, j) in enumerate(gnode_ids):
                 a_mtrx[i, j] += local_a_mtrx[I, J] * parent_mapping_jacobian

def assemble_load_vector(b_vec, domain_partition, parent_mapping, parent_basis_func_list, f,constrain=False):
    n = len(parent_basis_func_list[0])
    local_to_global_node_id_map = domain_partition[2]

    local_b_vec = np.zeros(n, dtype=np.float64)

    for (e, gnode_ids) in enumerate(local_to_global_node_id_map):
        (x_0, x_1) = domain_partition[0][e]
        h_e = x_1 - x_0
        parent_mapping_jacobian = parent_mapping[1](h_e)
        x_e_bar = (x_0 + x_1) / 2.0

        for I in range(n):
            parent_basis_func_I = parent_basis_func_list[0][I]
            f_parent = lambda zeta, x_e_bar=x_e_bar, h_e=h_e: f(parent_mapping[0](zeta, x_e_bar, h_e))
            integrand = lambda zeta: f_parent(zeta) * parent_basis_func_I(zeta)
            (local_b_vec[I], _) = scipy.integrate.quad(integrand, -1, 1)


        for (I, i) in enumerate(gnode_ids):
            b_vec[i] += local_b_vec[I] * parent_mapping_jacobian

def genCollocationPts(x, y, x_tilde_pts):
    slopeList = []
    interceptList = []
    for i in range(1, len(x)):
        slope = (y[i] - y[i - 1]) / (x[i] - x[i - 1])
        intercept = y[i] - (slope * x[i])
        slopeList.append(slope)
        interceptList.append(intercept)
    y_tilde_pts = []
    for i in range(len(x_tilde_pts)):
        result = np.where(x <= x_tilde_pts[i])
        x_tildeLoc = result[-1][-1]
        if x_tildeLoc >= len(slopeList):
            x_tildeLoc = len(slopeList) - 1
        y_tilde = x_tilde_pts[i] * slopeList[x_tildeLoc] + interceptList[x_tildeLoc]
        y_tilde_pts.append(y_tilde)
    return (np.asarray(y_tilde_pts))

In [ ]:
a=25
source_slope_value = 4.13e-4
l_2= 9.470
h = x_b
x_n=100


def source_slope(x,a,source_slope_value):
    source_s=np.zeros(x)
    i=0
    for i in range(x):
        source_s[i] = source_slope_value if (i<=a) else -1/(l_2**2)
    return source_s

x = np.linspace(x_a,x_b,100)
slope_x = source_slope(x_n,a,source_slope_value)

plt.figure(1, figsize=(14, 5))
plt.plot(x,slope_x,label = 'Source Slope')
plt.title(r'Source Slope s$(x)$',fontsize = 20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(loc='best', fontsize=12)
plt.grid(True)
plt.xlabel(r'$cm$')
plt.ylabel(r'$s(x)$')




In [ ]:
degree = 1
n_elem = 30 #Number of elements used


num_parts_list = [2, 5, 3]
locList = [x_a, a, x_b]

domain_partition = get_domain_partition(degree, n_elem, x_a, x_b)
parent_mapping = get_parent_mapping()
parent_basis_functions = get_parent_basis_functions()

phi_list = get_global_basis_functions(domain_partition, parent_mapping, parent_basis_functions, global_basis_function)[0]

phi_prime_list = get_global_basis_functions(domain_partition, parent_mapping, parent_basis_functions, global_basis_function)[1]

plot_func(domain_partition,phi_list,x_a,x_b,title=r'Lagrange Basis Functions (w/ Flux Boundary Conditions) (degree = %i; dim =%i'%(degree, len(phi_list))+')')

plot_func(domain_partition,phi_prime_list,x_a,x_b,title=r'Lagrange Basis Functions (w/ Flux Boundary Conditions) (degree = %i; dim =%i'%(degree, len(phi_list))+')')



In [ ]:
'''Build the coefficients of the lift function'''
import numpy as np
alpha_vec = np.zeros(len(phi_list))
alpha_vec[0] = u_a


In [ ]:
'''Build the lift function'''
# n_pts = 500
# x = np.linspace(x_a, x_b, n_pts)

# def w_lift(x):
#     if isinstance(x,(list,np.ndarray)):
#         a_mtrx = np.zeros((len(x),len(phi_list)))
#     elif isinstance(x,(int,float)):
#         a_mtrx = np.zeros((len(phi_list)))
#     for i, phi_i in enumerate(phi_list):
#         if isinstance(x,(list,np.ndarray)):
#             for j,x_j in enumerate(x):
#                 a_mtrx[j,i] = phi_i(x_j)
#         elif isinstance(x, (int, float)):
#             a_mtrx[i]=phi_i(x)
#     return a_mtrx@alpha_vec


In [ ]:
'''Build the lift function derivative'''

# def w_lift_prime(x):
#     a_mtrx = np.zeros(len(phi_prime_list))
#     for i, phi_i in enumerate(phi_prime_list):
#         a_mtrx[i] = phi_i(x)
#     return a_mtrx@alpha_vec

In [ ]:
'''Test boundary values'''

# print('w(a) = ', w_lift(x_a))

In [ ]:
'''Rayleigh Ritz Method with FEM Lagrange Basis Functions'''

# n_pts = 500
# x = np.linspace(x_a, x_b, n_pts)
# u_values = w_lift(x)


# import matplotlib.pyplot as plt
# #%matplotlib inline
# #plt.style.use('dark_background')
# plt.figure(1, figsize=(14, 5))

# plt.plot(x, u_values, 'r-', label='Lift function')

# plt.title(r'Rayleigh-Ritz Method with Finite Element Lagrange Basis Functions (n='+str(len(phi_list))+')', fontsize=20)
# plt.ylabel(r'$w(x)$', fontsize=18)
# plt.xlabel(r'$x$', fontsize=18)
# plt.xticks(fontsize=16)
# plt.yticks(fontsize=16)
# plt.legend(loc='best',fontsize=12)
# plt.grid(True)
# plt.show()

In [ ]:
'''Pedagogical inner product'''

from scipy.integrate import quad

def inner_product(u, v, patches):
    integrand = lambda x: u(x) * v(x)          
    inner_product = 0.0
    for nodes_x in patches:
        #print(nodes_x[0],nodes_x[1])
        (inner_product_e, _) = quad(integrand, nodes_x[0], nodes_x[1],limit=100)
        inner_product += inner_product_e           
    return inner_product

def source_slope(x,a,source_slope_value):
    return source_slope_value if (x<=a) else -1/(l_2**2)



In [ ]:
'''Build the matrix of coefficients of the linear system'''

import numpy as np
n = len(phi_list)
a_mtrx = np.zeros((n, n), dtype=np.float64)

# Diffusion
for i,phi_prime_i in enumerate(phi_prime_list):
    for j,phi_prime_j in enumerate(phi_prime_list):
        a_ij = inner_product(phi_prime_j, phi_prime_i,domain_partition[0])
        a_mtrx[i,j] = a_ij * diff_coeff


# Source        
for i,phi_i in enumerate(phi_list):
    for j,phi_j in enumerate(phi_list):
        s_x_phi_j = lambda x: source_slope(x,a,source_slope_value) * phi_j(x)
        a_ij = inner_product(s_x_phi_j, phi_i, domain_partition[0])
        a_mtrx[i,j] -= a_ij

In [ ]:
def inner_product_1(u, v, patches):
    integrand = lambda x: u(x,N,shift_fourier,kappa) * v(x)          
    inner_product = 0.0
    for nodes_x in patches:
        #print(nodes_x[0],nodes_x[1])
        (inner_product_e, _) = quad(integrand, nodes_x[0], nodes_x[1],limit=100)
        inner_product += inner_product_e           
    return inner_product

def Derivative2_FourierBasis_Mat(x,N,shift_fourier,Kappa):
    function = np.zeros((len([x]),2*N+1))
    for i in range (len([x])):
        for j in range((2*N+1)):
            if (j==0):
                function[i,j]=0
            elif (j%2)==1:
                function[i,j]=-(((((j//2)+1)*Kappa)**2)*np.cos(((j//2)*Kappa*(x-shift_fourier))))
            elif (j%2)==0:
                function[i,j]=-(((((j//2)+1)*Kappa)**2)*np.sin((j//2)*Kappa*(x-shift_fourier)))
    return np.array(function)

def FourierBasis_Mat(x,N,shift_fourier,Kappa):
    function=np.zeros((len([x]),2*N+1))
    for i in range (len([x])):
        for j in range((2*N+1)):
            if (j==0):
                function[i,j]=1
            elif (j%2)==1:
                function[i,j]=np.cos(((j//2)+1)*Kappa*(x-shift_fourier))
            elif (j%2)==0:
                function[i,j]=np.sin((j//2)*Kappa*(x-shift_fourier))
    return np.array(function)


In [ ]:
'''Build load vector'''

b_vec = np.zeros(n, dtype=np.float64)
q_nb = 0



for i, phi_i in enumerate(phi_list):
    # Source bias
    b_vec[i] = inner_product(source_bias, phi_i,domain_partition[0]) 
    # # Lift function: diffusion
    # term1 = inner_product(w_lift_prime, phi_prime_list[i],domain_partition[0])
    # b_vec[i] -= diff_coeff * term1
    # # Lift function: linear source
    # s_x_w = lambda x: source_slope(x,a,source_slope_value) * w_lift(x)
    # term2 = inner_product(s_x_w, phi_i,domain_partition[0])
    # b_vec[i] += term2
    # Neumann BC
    # b_vec[i] -= q_nb * phi_i(x_a)

print(b_vec)

In [ ]:
'''Compute optimal coefficient vector'''

c_star_vec = np.linalg.solve(a_mtrx, b_vec)

print(c_star_vec)

In [ ]:
'''Build the best approximation function in V_N'''

def femlb_evaluation_matrix(x):
    if (isinstance(x, list) or not isinstance(x, int) and isinstance(x, np.ndarray)):
        a = np.zeros((len(phi_prime_list),len(x)))
    elif isinstance(x,int):
        a = np.zeros((len(phi_list),1))
    for i, phi_i in enumerate(phi_list):
        a[i]=phi_i(x)
    a = np.transpose(a)
    return a

print()
def u_star(x):
    u_0 = femlb_evaluation_matrix(x)@c_star_vec
    #w = femlb_evaluation_matrix(x)@alpha_vec
    return u_0# + w

# def u_star(x):
#     return sum((c_star_vec[j])*phi_i(x) for (j,phi_i) in enumerate(phi_list))

In [ ]:
'''Build the best approximation function in V_N'''

def femlb_evaluation_matrix_prime(x):
    if (isinstance(x, list) or not isinstance(x, int) and isinstance(x, np.ndarray)):
        a = np.zeros((len(phi_prime_list),len(x)))
    elif isinstance(x, int):
        a = np.zeros((len(phi_prime_list),1))
    for i, phi_i in enumerate(phi_prime_list):
        a[i]=phi_i(x)
    a = np.transpose(a)
    return a

    
def u_prime_star(x):
    u_prime_0 = femlb_evaluation_matrix_prime(x)@c_star_vec
    #w_prime = femlb_evaluation_matrix_prime(x)@alpha_vec
    return u_prime_0# + w_prime

In [ ]:
'''Rayleigh Ritz Method with Fourier Basis Functions'''

n_pts = 500
x = np.linspace(x_a, x_b, n_pts)
u_values = u_star(x)
import matplotlib.pyplot as plt
#%matplotlib inline
#plt.style.use('dark_background')
plt.figure(1, figsize=(14, 5))

plt.plot(x, u_values, 'r-', label='Rayleigh-Ritz Solution w/ Dirichlet/Neumann BC')

plt.title(r'Rayleigh-Ritz Method with Finite Element Lagrange Basis Functions (n='+str(len(phi_list))+')', fontsize=20)
plt.ylabel(r'$u_N(x)$', fontsize=18)
plt.xlabel(r'$x$', fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(loc='best',fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
'''Rayleigh-Ritz solution evaluated at the boundaries'''

np.set_printoptions(precision=5)
print('u^*_N(a) = ',u_star(x_a), '  u^*_N(b) = ',u_star(x_b), " u^*'_N(b) = ",u_prime_star(x_b))

In [ ]:
'''Flux at boundary'''

print('q_nb computed = ', -diff_coeff*u_prime_star(x_b))
print('q_nb given    = ', q_nb)
print('flux error [%]= ',(-diff_coeff*u_prime_star(x_b) - q_nb)/q_nb*100)